In [1]:
import pandas as pd 
import numpy as np 
import matplotlib as plt 
%matplotlib inline 
import seaborn as sns 

In [2]:
df = pd.read_excel('../data/gamezone-orders-data.xlsx')
df.head()

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
0,2c06175e,0001328c3c220830,2020-12-24 00:00:00,2020-12-13,Nintendo Switch,e682,168.00,website,affiliate,unknown,US
1,ee8e5bc2,0002af7a5c6100772,2020-10-01 00:00:00,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,DE
2,9eb4efe0,0002b8350e167074,2020-04-21 00:00:00,2020-02-16,Nintendo Switch,8d0d,151.20,website,direct,desktop,US
3,cac7cbaf,0006d06b98385729,2020-04-07 00:00:00,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,AU
4,6b0230bc,00097279a2f46150,2020-11-24 00:00:00,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,TR


## Data Cleaning Plan

For this project, I'll be following the **CLEAN** approach to prepare the dataset for analysis:

- **Conceptualize the Data:** Understand what each feature represents and identify key columns.
- **Locate Solvable Problems:** Find and fix obvious issues like missing values, duplicates, and inconsistent formatting.
- **Evaluate Unsolvable Issues:** Address more complex problems such as outliers and unresolved missing data.
- **Augment the Data:** Create new features if they add value to the analysis.
- **Note and Document:** Record all cleaning steps and decisions for transparency.

This structured process will help ensure the data is reliable and ready for analysis.

## 1. Conceptualize

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21864 entries, 0 to 21863
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   USER_ID                  21864 non-null  object        
 1   ORDER_ID                 21864 non-null  object        
 2   PURCHASE_TS              21864 non-null  object        
 3   SHIP_TS                  21864 non-null  datetime64[ns]
 4   PRODUCT_NAME             21864 non-null  object        
 5   PRODUCT_ID               21864 non-null  object        
 6   USD_PRICE                21859 non-null  float64       
 7   PURCHASE_PLATFORM        21864 non-null  object        
 8   MARKETING_CHANNEL        21781 non-null  object        
 9   ACCOUNT_CREATION_METHOD  21781 non-null  object        
 10  COUNTRY_CODE             21826 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 1.8+ MB


- We have around 22k orders.

- We have just one metric or numeric features here and that's `USD_PRICE`, which is also our key features.
- More useful features could be: `PURCHASE_TS`, `SHIP_TS`M, `PRODUCT_NAME`, `MARKETING_CHANNEL`, `COUNTRY_CODE`.

- Possible type conversions: `PURCHASE_TS` to datetime.


## 2. Locate issues

### `USER_ID`

In [4]:
len(df['USER_ID'].unique()) / len(df) * 100

90.79308452250274

In [5]:
df['USER_ID'].duplicated().sum()

np.int64(2013)

In [6]:
2013/21864

0.09206915477497256

Almost 91% are unique user id, and only around 9% are repeating user ids in the dataset.

### `ORDER_ID`

In [7]:
len(df['ORDER_ID'].unique())

21719

In [8]:
df['ORDER_ID'].duplicated().sum()

np.int64(145)

We have 145 duplicate orders. We need to investigate this because there shouldn't be any duplicate orders in the data.

In [9]:
df[df['ORDER_ID'].duplicated() == True]['PRODUCT_NAME'].value_counts()

PRODUCT_NAME
Nintendo Switch              98
27in 4K gaming monitor       37
Sony PlayStation 5 Bundle    10
Name: count, dtype: int64

In [10]:
df[df['ORDER_ID'].duplicated() == True].head()

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
9379,b66cdb8d,7a5f67e18fa77291,2020-01-27 00:00:00,2020-01-28,27in 4K gaming monitor,e7e6,480.00,website,direct,desktop,US
9564,6270d6f9,7d09de332e342684,2020-01-27 00:00:00,2020-01-30,27in 4K gaming monitor,891b,332.20,website,direct,desktop,BR
9922,e80b93ad,815caec5eb998020,2020-01-30 00:00:00,2020-01-31,27in 4K gaming monitor,891b,408.20,website,email,desktop,DE
10052,3838f9e6,833086c869925765,2020-01-24 00:00:00,2020-01-25,27in 4K gaming monitor,891b,317.14,website,direct,desktop,JP
10142,fd7dd923,844a97334cd107082,2020-01-22 00:00:00,2020-01-25,27in 4K gaming monitor,891b,480.00,website,direct,desktop,US


Let's check the first duplicate.

In [11]:
df[df['ORDER_ID'] == '7a5f67e18fa77291']

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
9378,b66cdb8d,7a5f67e18fa77291,2020-01-27 00:00:00,2020-01-28,27in 4K gaming monitor,e7e6,480.0,website,direct,desktop,US
9379,b66cdb8d,7a5f67e18fa77291,2020-01-27 00:00:00,2020-01-28,27in 4K gaming monitor,e7e6,480.0,website,direct,desktop,US


Yes these are exact copies of each other.

I'm removing this duplicates right now.

In [12]:
df.drop_duplicates(subset='ORDER_ID', keep='first', inplace=True)

In [13]:
df.shape

(21719, 11)

### `PURCHASE_TS`:

In [14]:
df['PURCHASE_TS'].isna().sum()

np.int64(0)

In [15]:
# converting it to datetime
df['PURCHASE_TS'] = pd.to_datetime(df['PURCHASE_TS'], errors='coerce')

In [16]:
df['PURCHASE_TS'].isna().sum()

np.int64(5)

We have 5 missings. Let's check them.

In [17]:
df[df.PURCHASE_TS.isna()]

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
1047,a5298a4d,0dda212aaea69940,NaT,2019-07-08,JBL Quantum 100 Gaming Headset,ab0f,21.96,website,direct,desktop,FR
5846,a81bb521,4cd9ab100d971208,NaT,2021-01-11,Nintendo Switch,8d0d,120.26,website,direct,desktop,IE
11853,2fa9f33d,99d824517da22388,NaT,2019-04-11,JBL Quantum 100 Gaming Headset,ab0f,21.19,website,direct,mobile,JP
16163,b313cea5,c9e0aea0d9a75871,NaT,2019-05-18,JBL Quantum 100 Gaming Headset,ab0f,19.20,website,direct,desktop,US
20725,67f8050b,f4de38506b644875,NaT,2019-01-17,JBL Quantum 100 Gaming Headset,ab0f,25.69,website,direct,desktop,GB


In [18]:
print("Start date: ", df['PURCHASE_TS'].min())
print("End date: ", df['PURCHASE_TS'].max())

Start date:  2019-01-01 00:00:00
End date:  2021-02-28 00:00:00


### `SHIP_TS`

In [19]:
df['SHIP_TS'].isna().sum()

np.int64(0)